In [2]:
import sentencepiece as spm


In [3]:
vocab_sizes = [8000, 16000, 32000]

for size in vocab_sizes:
    spm.SentencePieceTrainer.train(
        input='/home/rana/Projects/zindi/data/all_dyu.txt',
        model_prefix=f'dyula_{size}',
        vocab_size=size,
        character_coverage=1.0,
        model_type='bpe',
        input_sentence_size=1000000,
        shuffle_input_sentence=True
    )

    # Load the model and test it on some sample text
    sp = spm.SentencePieceProcessor()
    sp.load(f'dyula_{size}.model')
    
    sample_text = "an ni sɔgɔma hɛrɛ sira somɔgɔw do?"
    print(f"Vocab size: {size}")
    print(f"Tokenized: {sp.encode(sample_text, out_type=str)}")
    print()

In [ ]:
vocab_sizes = [8000, 16000, 32000]

for size in vocab_sizes:
    spm.SentencePieceTrainer.train(
        input='/home/rana/Projects/zindi/data/all_fr.txt',
        model_prefix=f'fr_{size}',
        vocab_size=size,
        character_coverage=1.0,
        model_type='bpe',
        input_sentence_size=1000000,
        shuffle_input_sentence=True
    )

    # Load the model and test it on some sample text
    sp = spm.SentencePieceProcessor()
    sp.load(f'fr_{size}.model')
    
    sample_text = "Votre opérateur téléphonique local devrait pouvoir vous en dire plus au sujet de la connexion à ce service."
    print(f"Vocab size: {size}")
    print(f"Tokenized: {sp.encode(sample_text, out_type=str)}")
    print()

In [ ]:

# Train Dyula SentencePiece model
spm.SentencePieceTrainer.train(
    input='/home/rana/Projects/zindi/data/all_dyu.txt',
    model_prefix='dyu',
    vocab_size=32000,
    character_coverage=1.0,
    model_type='bpe',
    input_sentence_size=1000000,
    shuffle_input_sentence=True
)

# Train French SentencePiece model
spm.SentencePieceTrainer.train(
    input='french_sentences.txt',
    model_prefix='french',
    vocab_size=32000,
    character_coverage=1.0,
    model_type='bpe',
    input_sentence_size=1000000,
    shuffle_input_sentence=True
)